In [11]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, transpile
from qiskit.circuit.library import PhaseEstimation
from qiskit.circuit.library.arithmetic import ExactReciprocal, PiecewiseChebyshev
from qiskit.providers import Backend

class HHL:
    """
    A simplified HHL algorithm for solving Ax = b, adapted for Qiskit 1.4.2.
    This version accepts a backend and uses backend.run() instead of execute().
    """
    def __init__(self, epsilon: float = 1e-2, backend: Backend = None,
                 use_exact_reciprocal: bool = True) -> None:
        self.epsilon = epsilon
        self._epsilon_r = epsilon / 3  # tolerance for controlled rotation
        self._epsilon_s = epsilon / 3  # tolerance for state preparation
        self._epsilon_a = epsilon / 6  # tolerance for Hamiltonian simulation
        self.use_exact_reciprocal = use_exact_reciprocal
        self.scaling = 1.0
        self.backend = backend  # a Backend instance

    def _get_delta(self, n_l: int, lambda_min: float, lambda_max: float) -> float:
        formatstr = "#0" + str(n_l + 2) + "b"
        lambda_min_tilde = np.abs(lambda_min * (2**n_l - 1) / lambda_max)
        if np.abs(lambda_min_tilde - 1) < 1e-7:
            lambda_min_tilde = 1
        binstr = format(int(lambda_min_tilde), formatstr)[2:]
        return sum(int(bit) / (2 ** (i + 1)) for i, bit in enumerate(binstr))

    def construct_circuit(self, matrix, vector, neg_vals: bool = True) -> QuantumCircuit:
        # --- State Preparation ---
        if isinstance(vector, QuantumCircuit):
            nb = vector.num_qubits
            vector_circuit = vector
        elif isinstance(vector, (list, np.ndarray)):
            vector = np.array(vector, dtype=complex)
            nb = int(np.log2(len(vector)))
            vector_circuit = QuantumCircuit(nb)
            vector_circuit.initialize(vector / np.linalg.norm(vector), list(range(nb)))
        else:
            raise ValueError("Invalid type for vector. Must be list, ndarray, or QuantumCircuit.")
        
        # --- Hamiltonian Simulation ---
        if isinstance(matrix, QuantumCircuit):
            matrix_circuit = matrix
        elif isinstance(matrix, (list, np.ndarray)):
            matrix = np.array(matrix, dtype=complex)
            if matrix.shape[0] != matrix.shape[1]:
                raise ValueError("Matrix must be square.")
            if np.log2(matrix.shape[0]) % 1 != 0:
                raise ValueError("Matrix dimension must be a power of 2.")
            if not np.allclose(matrix, matrix.conj().T):
                raise ValueError("Matrix must be Hermitian.")
            if matrix.shape[0] != 2 ** nb:
                raise ValueError("Dimension mismatch between vector and matrix.")
            # Implementing Hamiltonian simulation for a NumPy matrix is non-trivial.
            raise NotImplementedError("Hamiltonian simulation for a NumPy matrix is not implemented here.")
        else:
            raise ValueError("Invalid type for matrix.")
        
        # --- Eigenvalue Estimation Setup ---
        n_l = 3  # number of eigenvalue qubits; adjust as needed
        delta = 1.0  # placeholder; in a full implementation compute from eigenvalue bounds

        # --- Reciprocal (Controlled Rotation) ---
        if self.use_exact_reciprocal:
            reciprocal_circuit = ExactReciprocal(n_l, delta, neg_vals=neg_vals)
            na = reciprocal_circuit.num_ancillas
        else:
            degree = 2  # example degree; adjust as needed
            breakpoints = [2, 3]  # example breakpoints; adjust as needed
            reciprocal_circuit = PiecewiseChebyshev(lambda x: np.arcsin(delta / x),
                                                    degree, breakpoints, n_l)
            na = reciprocal_circuit.num_ancillas

        # --- Register Initialization ---
        qb = QuantumRegister(nb, name='qb')  # state register for |b> and solution
        ql = QuantumRegister(n_l, name='ql')  # eigenvalue register
        qa = AncillaRegister(na, name='qa') if na > 0 else None  # ancilla for reciprocal
        qf = QuantumRegister(1, name='qf')  # flag qubit

        if qa is not None:
            qc = QuantumCircuit(qb, ql, qa, qf)
        else:
            qc = QuantumCircuit(qb, ql, qf)

        # --- Build the Circuit ---
        # 1. Prepare state |b>
        qc.append(vector_circuit, qb[:])
        # 2. Phase Estimation for Hamiltonian simulation
        phase_est = PhaseEstimation(num_evaluation_qubits=n_l, unitary=matrix_circuit)
        if qa is not None:
            qc.append(phase_est, ql[:] + qb[:] + qa[:])
        else:
            qc.append(phase_est, ql[:] + qb[:])
        # 3. Controlled rotation using the reciprocal function
        if self.use_exact_reciprocal:
            qc.append(reciprocal_circuit, ql[::-1] + [qf[0]])
        else:
            qc.append(reciprocal_circuit.to_instruction(), ql[:] + [qf[0]] + (qa[:] if qa is not None else []))
        # 4. Inverse Phase Estimation
        if qa is not None:
            qc.append(phase_est.inverse(), ql[:] + qb[:] + qa[:])
        else:
            qc.append(phase_est.inverse(), ql[:] + qb[:])
        
        qc = transpile(qc, optimization_level=3)
        return qc

    def solve(self, matrix, vector):
        qc = self.construct_circuit(matrix, vector)
        # Use provided backend or default to a statevector simulator
        if self.backend is None:
            from qiskit_aer import AerSimulator
            backend = AerSimulator(method="statevector")
        else:
            backend = self.backend

        compiled_qc = transpile(qc, backend)
        compiled_qc.save_statevector()
        job = backend.run(compiled_qc)
        result = job.result()
        try:
            state = result.get_statevector(compiled_qc)
            return state, compiled_qc
        except Exception:
            return result, compiled_qc



In [12]:
A = np.array([[3, 1],
              [1, 2]], dtype=float)
b = np.array([9, 8], dtype=float)

# Solve the linear system A x = b using NumPy's solver
x = np.linalg.solve(A, b)

print("The solution x is:", x)

The solution x is: [2. 3.]


In [13]:
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import AerSimulator
from scipy.linalg import expm

backend = AerSimulator(method="statevector")
hhl = HHL(backend=backend)

t=1
U = expm(-1j * A * t)
matrix_circ = QuantumCircuit(1)
matrix_circ.append(UnitaryGate(U), [0])

matrix = A  # Define your matrix (as a QuantumCircuit for simulation)
vector = b  # Example vector for a 1-qubit system
solution_state, compiled_qc = hhl.solve(matrix_circ, vector)
print(solution_state)

Statevector([ 7.47409319e-01-5.68153666e-16j,
              6.64363839e-01-3.87763577e-16j,
              8.60026734e-18+1.86790333e-16j,
             -2.95879573e-16+1.43754410e-16j,
             -1.38268256e-16-1.58961853e-17j,
              1.98347626e-16+6.88595267e-17j,
              2.22507192e-16-2.47134628e-16j,
              9.59856379e-17+4.00857272e-16j,
             -2.42657025e-16-1.75222028e-16j,
             -8.66340428e-17-4.29110636e-17j,
             -1.31840879e-16-3.73716777e-16j,
             -5.44424352e-17+4.43918860e-16j,
             -2.08267915e-16-2.26296501e-18j,
             -6.60155098e-17-1.18881267e-16j,
             -1.57158783e-16+8.41101476e-17j,
              3.37603098e-16-1.35310680e-16j,
             -1.66533454e-16-6.85533053e-17j,
              1.66533454e-16-5.46966695e-17j,
              6.84394481e-17+5.12288147e-17j,
             -1.69662492e-17+5.85032265e-19j,
             -1.12232728e-17+1.26192347e-16j,
             -5.16038395e-17+1.026

In [19]:
n_b = 1    # your system has 1 qubit if A is 2x2
n_l = 3    # you used 3 phase-estimation qubits
n_a = 0    # or however many ancillas the reciprocal uses (ExactReciprocal often uses 0 for 1-qubit system)
n_f = 1    # the single "flag" qubit

total_qubits = n_b + n_l + n_a + n_f  # e.g. = 1 + 3 + 0 + 1 = 5
reshaped = np.reshape(solution_state.data, [2]*total_qubits)
ql_zero_slice = 0 if n_l == 1 else (0,)*n_l  
qa_zero_slice = 0 if n_a == 1 else (0,)*n_a
solution_amplitudes = reshaped[:, *(0,)*n_l, *(0,)*n_a, 1]
norm = np.linalg.norm(solution_amplitudes)
if norm > 1e-12:
    solution_amplitudes = solution_amplitudes / norm

print("Raw solution vector (as a quantum state) =", solution_amplitudes)

Raw solution vector (as a quantum state) = [1.00000000e+00-5.83661473e-16j 2.50666042e-16-8.23293899e-17j]
